<a href="https://colab.research.google.com/github/SatyaKrishna/GssMILP-Anomaly-Classification-in-Surveillance-Videos/blob/main/GssMILP_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/NSK

/content/drive/My Drive/NSK


In [3]:
import sys
import cv2
import numpy as np
import os
import os.path
import cvxpy as cp
import collections as C
from os import path
import math
import random
from datetime import datetime
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import csgraph_from_dense
from scipy.sparse import csgraph
from scipy.sparse import identity
from scipy.sparse.linalg import inv
from notebook import notebookapp
notebookapp.iopub_data_rate_limit= 10000000.0
notebookapp.rate_limit_window=12.0

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


In [ ]:
'''
class Vertex:
  def __init__(self, node):
    self.id = node
    self.adjacent = {}
  
  def __str__(self):
    return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])
  
  def add_neighbor(self, neighbor, weight=0.0):
    self.adjacent[neighbor] = weight
  
  def get_connections(self):
    return self.adjacent.keys()
  
  def get_id(self):
    return self.id
  
  def get_weight(self, neighbor):
    return self.adjacent[neighbor]

In [ ]:
'''
class Graph:
  def __init__(self):
    self.vert_dict = {}
    self.num_vertices = 0
  
  def __iter__(self):
    return iter(self.vert_dict.values())
  
  def add_vertex(self, node):
    self.num_vertices = self.num_vertices + 1
    new_vertex = Vertex(node)
    self.vert_dict[node] = new_vertex
    return new_vertex
  
  def get_vertex(self, n):
    if n in self.vert_dict:
      return self.vert_dict[n]
    else:
      return None
  
  def add_edge(self, frm, to, cost = 0.0):
    if frm not in self.vert_dict:
      self.add_vertex(frm)
    if to not in self.vert_dict:
      self.add_vertex(to)
    
    self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
    self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

  def get_vertices(self):
    return self.vert_dict.keys()

In [5]:
def find_segment_level_labels(frame_level_labels_file, root_path_for_videos_directory, output_file,data_set):
  """
  This function finds the video-segment level labels from frame level labels. The output will written in a text file in the following format.
  "video_segment_index label
  video_segment_index label
  video_segment_index label
        ...
        ...
  video_segment_index label"
  
  video_segment_index is a string consist three parts. i) v_type: video type ii) v_name: video name iii) s_index: segment index
  label is either 1/-1
  
  """
  f = open(frame_level_labels_file,"r").readlines()
  of = open(output_file,"w")
  
  for line in f:
    # Ped1 eample # Abnormal_Test024_video.avi 50 160
    # Ped2 example # Abnormal_Test001_video.avi 61 180
    # MEM example # Abnormal_012.mp4 200 500 616 930
    # WS example # Abuse003_x264.mp4 2148 2196
    words = line.strip().split(' ')
    if data_set == 'Ped1':
      v_name = words[0] # Abnormal_Test024.mp4
      v_type = v_name.split('_')[0] # Abnormal
      v_name = v_name.replace(v_type+'_','') # Test024_video.avi
      v_name = v_name.replace(".avi",".txt")
    if data_set == 'Ped2':
      v_name = words[0] # Abnormal_Test001_video.avi
      v_type = v_name.split('_')[0] # Abnormal
      v_name = v_name.replace(v_type+'_','') # Test001_video.avi
      v_name = v_name.replace(".avi",".txt") # Test001_video.txt      
    if data_set == 'MEM':
      v_name = words[0] # Abnormal_012.mp4
      v_type = v_name.split('_')[0] # Abnormal
      v_name = v_name.replace(v_type+'_','') # 012.mp4
      v_name = v_name.replace(".mp4",".txt") # 012.txt
    if data_set == 'WS':
      v_name = words[0] # Abuse003_x264.mp4
      if 'Normal' in v_name:
        v_type = 'Normal'
      else:
        v_type = 'Abnormal'
      v_name = v_name.replace(".mp4",".txt") # Abuse003_x264.txt

    if int(words[1]) == -1 and int(words[2]) == -1 :
      for i in range(32):
        string = "{}_{}_{} {}\n".format(v_type,v_name,i,-1)
        of.write(string)
    else:
      length = len(words)
      no_of_frames = 0
      # the following statement take more time while running the code in colab
      #no_of_frames = len(glob.glob(root_path_for_video_frames_directory+v_type+"/"+v_name.replace("_C.txt","")+"/"+"*.jpg"))
      if data_set == 'Ped1':
        v_path = root_path_for_videos_directory+v_type+'/'+v_name.replace('.txt','avi')
        cap = cv2.VideoCapture(v_path)
        no_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        no_of_frames = int(no_of_frames)
        cap.release()
      if data_set == 'Ped2':
        v_path = root_path_for_videos_directory+v_type+'/'+v_name.replace('.txt','avi')
        cap = cv2.VideoCapture(v_path)
        no_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        no_of_frames = int(no_of_frames)
        cap.release()
      if data_set == 'MEM':
        v_path = root_path_for_videos_directory+v_type+'/'+v_name.replace('.txt','.mp4')
        cap = cv2.VideoCapture(v_path)
        no_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        no_of_frames = int(no_of_frames)
        cap.release()
      if data_set == 'WS':
        v_path = root_path_for_videos_directory+v_type+'/'+v_name.replace('.txt','.mp4')
        cap = cv2.VideoCapture(v_path)
        no_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        no_of_frames = int(no_of_frames)
        cap.release()
      if no_of_frames <= 1024:
        no_of_frames = no_of_frames + (1056 - no_of_frames)

      line_space_list = np.linspace(0,no_of_frames,33)
      line_space_list = [int(round(x)) for x in line_space_list]
      for i in range(32):
        string = "{}_{}_{} {}\n".format(v_type,v_name,i,-1)
        SS = line_space_list[i]+1
        SE = line_space_list[i+1]-1
        
        w = 1
        while w < length:
          if SS <= int(words[w]) and int(words[w+1]) <= SE:
            string = "{}_{}_{} {}\n".format(v_type,v_name,i,1)
            break
          elif SS >= int(words[w]) and SS < int(words[w+1]) and SE >= int(words[w+1]):
            string = "{}_{}_{} {}\n".format(v_type,v_name,i,1)
            break
          elif SS < int(words[w]) and SE > int(words[w]) and SE <= int(words[w+1]):
            string = "{}_{}_{} {}\n".format(v_type,v_name,i,1)
            break
          elif SS >= int(words[w]) and SE <= int(words[w+1]) :
            string = "{}_{}_{} {}\n".format(v_type,v_name,i,1)
            break
          else:
            w = w+2
        of.write(string)
  of.close()

In [ ]:
# Following lines for testing the above function
#frame_level_labels_file = "./dataset/Temporal_Anomaly_Annotations_for_full_data_of_ped2.txt"
#root_path_for_videos_directory = './AllVideos/UCSD_Ped2/'
#output_file = "./dataset/segment_level_labels_ped2.txt"
#data_set = 'Ped2'
#find_segment_level_labels(frame_level_labels_file, root_path_for_videos_directory, output_file,data_set)

In [6]:
def combining_video_features(videos_path,segment_level_labels_file_path, feat_option):
  """
    This function combines the all types of video featues in to a single list of list datastructure as shown below:
    [ [key, feature_vector, label],
      [key, feature_vector, label],

      [key, feature_vector, label] ]. 
    
    key is a string type. it contains three parts. 1: first part is the type of video, 
                                                   2. second part denotes the name of the video and 
                                                   3. Third part denotes the index of video segment. 
    Ex: Arrest_Arrest001_x264_C.txt_0. For each index key it contains a list of 4096 features and one label
    feature_vector: it is a list of 4096 float numbers
    label: it is an integer

    arguments: 
    1. videos_path: it will take the root path of all types of anomaly folders in which we have one text file of 32 features for each video 
    Ex: videos_path="/media/cloud/data2/AnomalyDetectionCVPR2018/C3D-features-for-Training-and-Testing-on-WS/Train/"
    2. segment_level_labels_file_path: 
    Ex: "/media/cloud/data2/AnomalyDetectionCVPR2018/ADVS-new-code/dataset/segment_level_labels_WS.txt"
  """
  
  local_dataset = []
  local_dataset_labels = {}
  
  f = open(segment_level_labels_file_path,"r").readlines()
  for line in f:
    w = line.strip().split(' ')
    local_dataset_labels[w[0]] = int(w[1])
  
  abnormal_types = os.listdir(videos_path+'/'+'1'+'/')
  #print(abnormal_types)
  for v_type in abnormal_types:
    videos_list = os.listdir(videos_path+'/'+'1'+'/'+v_type+'/')
    #print(videos_list)
    for v_name in videos_list:
      #print(v_name)
      if feat_option == 1:
        f = open(videos_path+'/'+'1'+'/'+v_type+'/'+v_name,'r').readlines()
        for s_index in range(32):
          #local_dataset[v_type+"_"+v_name+"_"+str(s_index)] = f[s_index].strip().split(' ')  # this statement is used if
          local_dataset.append([v_type+"_"+v_name+"_"+str(s_index), [float(e) for e in f[s_index].strip().split(' ')],  local_dataset_labels[v_type+"_"+v_name+"_"+str(s_index)] ])
        #f.close()
      if feat_option == 2:
        if not (path.exists(videos_path+'/'+'2'+'/'+v_type+'/'+v_name) and path.exists(videos_path+'/'+'3'+'/'+v_type+'/'+v_name) and path.exists(videos_path+'/'+'4'+'/'+v_type+'/'+v_name) ):
          print('FileNotExists - ERROR:')
          print(videos_path+'/'+'2'+'/'+v_type+'/'+v_name)
          print(videos_path+'/'+'3'+'/'+v_type+'/'+v_name)
          print(videos_path+'/'+'4'+'/'+v_type+'/'+v_name)
          sys.exit()
        f2 = open(videos_path+'/'+'2'+'/'+v_type+'/'+v_name,'r').readlines()
        f3 = open(videos_path+'/'+'3'+'/'+v_type+'/'+v_name,'r').readlines()
        f4 = open(videos_path+'/'+'4'+'/'+v_type+'/'+v_name,'r').readlines()
        for s_index in range(32):
          l2 = [float(e) for e in f2[s_index].strip().split(' ')]
          l3 = [float(e) for e in f3[s_index].strip().split(' ')]
          l4 = [float(e) for e in f4[s_index].strip().split(' ')]
          local_dataset.append([v_type+"_"+v_name+"_"+str(s_index), l2 + l3 + l4,  local_dataset_labels[v_type+"_"+v_name+"_"+str(s_index)] ])
        #f2.close()
        #f3.close()
        #f4.close()
      if feat_option == 3:
        if not (path.exists(videos_path+'/'+'1'+'/'+v_type+'/'+v_name) and path.exists(videos_path+'/'+'2'+'/'+v_type+'/'+v_name) and path.exists(videos_path+'/'+'3'+'/'+v_type+'/'+v_name) and path.exists(videos_path+'/'+'4'+'/'+v_type+'/'+v_name) ):
          print('FileNotExists - ERROR:')
          print(videos_path+'/'+'1'+'/'+v_type+'/'+v_name)
          print(videos_path+'/'+'2'+'/'+v_type+'/'+v_name)
          print(videos_path+'/'+'3'+'/'+v_type+'/'+v_name)
          print(videos_path+'/'+'4'+'/'+v_type+'/'+v_name)
          sys.exit()
        f1 = open(videos_path+'/'+'1'+'/'+v_type+'/'+v_name,'r').readlines()
        f2 = open(videos_path+'/'+'2'+'/'+v_type+'/'+v_name,'r').readlines()
        f3 = open(videos_path+'/'+'3'+'/'+v_type+'/'+v_name,'r').readlines()
        f4 = open(videos_path+'/'+'4'+'/'+v_type+'/'+v_name,'r').readlines()
        for s_index in range(32):
          l1 = [float(e) for e in f1[s_index].strip().split(' ')]
          l2 = [float(e) for e in f2[s_index].strip().split(' ')]
          l3 = [float(e) for e in f3[s_index].strip().split(' ')]
          l4 = [float(e) for e in f4[s_index].strip().split(' ')]
          local_dataset.append([v_type+"_"+v_name+"_"+str(s_index), l1 + l2 + l3 + l4,  local_dataset_labels[v_type+"_"+v_name+"_"+str(s_index)] ])
        #f1.close()
        #f2.close()
        #f3.close()
        #f4.close()
        
  return local_dataset,local_dataset_labels

In [ ]:
# Following lines for testing the above function
#videos_path='./dataset/Ped2_dataset_1'
#segment_level_labels_file_path = './dataset/segment_level_labels_ped2.txt'
#features_option = 3
#dataset, labels = combining_video_features(videos_path, segment_level_labels_file_path,features_option)
#print("Dataset Inf: Number of instances - {}, Number of labes {}".format(len(dataset),len(labels)))
#print('Lenth of feature vector: {}'.format(len(dataset[0][1])))
#print("A DICTIONARY OF FEATURES AND A DICTIONARY OF LABELS ARE CREATED.\n\n")

In [7]:
# Python program to print connected  
# components in an undirected graph 
class Graph: 
      
    # init function to declare class variables 
    def __init__(self,V): 
        self.V = V 
        self.adj = [[] for i in range(V)] 
  
    def DFSUtil(self, temp, v, visited): 
  
        # Mark the current vertex as visited 
        visited[v] = True
  
        # Store the vertex to list 
        temp.append(v) 
  
        # Repeat for all vertices adjacent 
        # to this vertex v 
        for i in self.adj[v]: 
            if visited[i] == False: 
                  
                # Update the list 
                temp = self.DFSUtil(temp, i, visited) 
        return temp 
  
    # method to add an undirected edge 
    def addEdge(self, v, w): 
        self.adj[v].append(w) 
        self.adj[w].append(v) 
  
    # Method to retrieve connected components 
    # in an undirected graph 
    def connectedComponents(self): 
        visited = [] 
        cc = [] 
        for i in range(self.V): 
            visited.append(False) 
        for v in range(self.V): 
            if visited[v] == False: 
                temp = [] 
                cc.append(self.DFSUtil(temp, v, visited)) 
        return cc

In [8]:
# Graph Construction
def graph_construction(neighbors,dataset):
  neighbors=neighbors
  time_before = datetime.now() # starting time for graph construction
  X = []
  for i in range(len(dataset)):
    X.append(dataset[i][1])
  X = np.array(X)
  nbrs_graph = NearestNeighbors(n_neighbors=neighbors, algorithm='ball_tree', metric = 'euclidean',n_jobs = 24).fit(X)  #keyword argument 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']
  distances, indices = nbrs_graph.kneighbors(X)
  print("Time taken for K-NN graph construction: {}".format(datetime.now() - time_before))
  return distances, indices

In [9]:
#import numpy as np
#import cvxpy as cp
#import collections as C

#def NCGssMILP(W, y,  Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N):
#def NCGssMILP(GL,  Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N):
def NCGssMILP(W,GL, indices, Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N):
  """
  W : weighted similarity matrix of size 'n x n'
  y : actual bag-level labels
  Iy : List of lists contains the instance-level actual labels
  lambda1 : hyperperameter used as the coefficient of term-2
  lambda2a : hyperperameter used as the coefficient of term-3
  lambda2b : hyperperameter used as the coefficient of term-4
  lambda3 : hyperperameter used as the coefficient of term-5
  L : Total number of labeled bags
  PL : Total number of positive labeled bags
  NL : Total number of negative labeled bags
  n : Total number of instances
  N : Toatal number of bags

  Output : y_star belongs to {+1,-1}
  """
  threshold_value = 0.000001
  # Cmputing total positive labeled instances and negative labeled instances
  # Converting bag level labels to Instance-level labels
  nplI =  PL * 32
  nnlI = NL * 32
  #print('No.of positive labeled instances : {}'.format(nplI))
  #print('No.of negative labeled instances : {}'.format(nnlI))

  # computation of hyperparameters and initial value of f
  alpha = n/(2 * (nplI + nnlI))
  # computation of F* = Invers((I - alpha S))*Y
  IY = csr_matrix(np.reshape(np.array(Iy),[n,1]))
  f_star = np.squeeze(inv(identity(n) - GL.multiply(alpha)).dot(IY).toarray())
  g = []
  for i in range(n):
    if f_star[i] >= 0.0:
      g.append(1.0)
    else:
      g.append(-1.0)
  g = np.array(g)
  beta = np.dot(np.dot(g.transpose(),L),g)

  lambda3 = (2.0 * beta) / (n - (nplI + nnlI))
  lambda2a = (nnlI * alpha)/ (nplI + nnlI)
  lambda2b = (nplI * alpha)/ (nplI + nnlI)

  # Initial guess f_zero
  #f_zero = np.random.rand(n)
  #f_zero = f_zero * 2.0 - 1.0
  
  f_zero = f_star
  
  # Initialization of f_new
  f_new = np.array(list(f_zero))
  #xi = np.random.uniform(size=PL)
  xi = cp.Variable(PL)
  Iteration = 0
  while Iteration <= 50:
    f_old = np.array(list(f_new))
    f = cp.Variable(n)
    #xi = cp.Variable(PL)
    #xi = np.random.uniform(size=PL)
    #print('xi value : {}'.format(xi))

    # code for term1 f^TLf
    #expr1 = cp.quad_form(f, GL)
    
    #expr1 = np.dot(np.dot(cp.reshape(f,[1,n]),GL),f)
    
    #temp_expr1 = cp.multiply(cp.reshape(f,[1,n]) , GL)
    #expr1 = cp.multiply(temp_expr1 , f)
    
    #temp_expr1 = GL*f
    #expr1 = temp_expr1.T * f
    
    #temp_expr1 = cp.atoms.affine.transpose.transpose(f)*GL
    #expr1 = temp_expr1*f
    #Sum = 0.0
    #for i in range(n):
    #  i_neighbours_indices = []
    #  for j in range(number_of_neighbours):
    #    i_neighbours_indices.append(indices[i][j])
    #  
    #  for neighbour_index in i_neighbours_indices:
    #    Sum = Sum + W[i][neighbour_index] * cp.square(f[i] - f[neighbour_index])
    #expr1 = Sum / n
    Sum = 0.0
    for i in range(n):
     for j in indices[i]:
       Sum = Sum + W[i][j] * cp.square(f[i] - f[j])
    expr1 = Sum / n

    

    # code for term2 : temporal smoothness
    expr2 = lambda1 * cp.sum_squares(f[nplI:nplI+nnlI] - f[nplI+1:nplI+nnlI+1])

    # code for term3 : meansquare error over negative instances 
    expr3 = lambda2a * cp.sum_squares(f[nplI:nplI+nnlI] + np.ones(nnlI)) 

    # code for term4 : meansquare error meansquare error over positive instances
    expr4 = lambda2b * cp.sum_squares(xi)

    # code for term5 : 
    expr5 = lambda3 * cp.sum(cp.multiply(f[nplI+nnlI:n],f_old[nplI+nnlI:n]))   
    
    #obj_function = cp.Minimize(expr1+expr2+expr3+expr4+expr5)
    obj_function = cp.Minimize(expr1+expr2+expr3+expr4-expr5)

    # code for constraints
    # code for constraint1
    const1 = [f >= -1.0, f <= 1.0]

    # code for constraint2
    # 1. reshape the f_old into list of lists
    # 2. compute delta matrix and max vector for positive bags
    # 3. define constraint2

    #f_old_lists = f_old.reshape((-1,32))
    f_old_lists = f_old.reshape((-1,32))[0:PL,] # selecting only positive labeled instances
    #print('Shape : {}'.format(f_old_lists.shape))
    number_of_bags = np.shape(f_old_lists)[0]
    if number_of_bags == PL:
      #print("number of bags are matched")
      pass
    else:
      print("      - Logical error: Number of bags are not matched")
    zeta = np.max(f_old_lists,axis=1) # list of maximum label score from each positive bag
    #print('zeta : {}'.format(zeta))

    maximum_value_counts = [C.Counter(list(f_old_lists[i]))[zeta[i]] for i in range(len(zeta))] # finding count of maximum label score in each bag
    #print(maximum_value_counts)
    
    delta = np.zeros((PL,32))
    for r_index in range(PL):
      max_value = zeta[r_index]
      for c_index in range(32):
        if f_old_lists[r_index,c_index] == max_value:
          delta[r_index,c_index] = 1.0/maximum_value_counts[r_index]
    #print('reshape output - : {}'.format(cp.reshape(f,[32,N])[0:32,0:PL]))
    #print('f_old[0:PL] : {}'.format(f_old[0:PL]))
    #print('Shape of Delta is : {}'.delta.shape)
    #print('Delta : \n {}'.format(delta))
    
    temp_expr = np.dot(delta, cp.reshape(f,[32,N])[0:32,0:PL] - cp.reshape(f_old,[32,N])[0:32,0:PL])
    temp1 = temp_expr.diagonal()
    temp2 = np.subtract(np.subtract(np.ones(PL), zeta),temp1)
    const2 = [temp2[i] <= xi[i] for i in range(PL)] #temp2 <= xi


    # code for constraint3
    const3 = [xi >= 0.0]

    #constraints = const1 + const2 + const3
    constraints = const1+const2+const3
    prob = cp.Problem(obj_function, constraints)
    #prob.solve(solver=cp.SCS, use_indirect=True)
    #prob.solve(solver=cp.OSQP, verbose=True,max_iter = 35000)
    prob.solve(solver=cp.OSQP, max_iter = 185000)
    f_new = f.value
    Norm_value = cp.norm((f_new - f_old),1).value
    print('      Iteration No : {} \t Norm value : {}'.format(Iteration, Norm_value))
    if  Norm_value <= threshold_value :
      break
    Iteration = Iteration + 1
  return f_new

In [10]:
def Labeled_and_Unlabeled_datapreparation(DoS, videos_path, segment_level_labels_file_path, dataset, labels):
    """
        This function selects the training(labeled) videos randomly from each type of videos with uniform distrbution based on DoS value and then 
        the rest are considered as unlabeled data. For GssMILP, the features are arranged as D = {L^{+}, L^{-}, U}.
        Here, D : set of features
              L^{+} : subset of positive labeled data points
              L^{-} : subset of negative labeled data points
              U : subset of Unlabeled data points
              L = L^{+} + L^{-} : number of labeled points
              U = number of Unlabeled points
              L+U = N : total number of data points
                
    arguments:
            DoS: It is float data type. It denotes the percentage of labeld data to be consider for supervision.
            videos_path: It is string type, denotes the dataset path, in which we have a folder for each type of videos and named with integer numbers.
                    Ex: dataset_ptah = "/media/cloud/data2/AnomalyDetectionCVPR2018/C3D-features-for-Training-and-Testing-on-WS/Train/"
            segment_level_labels_file_path: "/media/cloud/data2/AnomalyDetectionCVPR2018/ADVS-new-code/dataset/segment_level_labels_WS.txt"
            #no_abnormal_types: It is integer type. It indicates the how many different types (classes) of videos considered.   
    """
    totalInstances = len(dataset)
    totalNumberOfVideos = int(totalInstances/32)
    
    # finding list of different abnormal type of videos exist in dataset
    abnormal_types = os.listdir(videos_path+'/'+'1'+'/')
    no_abnormal_types = len(abnormal_types)
    print("   Number of abnormal types in dataset is: {}".format(len(abnormal_types)))
    
    
    # Calculation of no of videos have to be select from each class and  total no.of labeled videos
    nlv = 0
    nlv_per_anomaly_type = {}
    for video_type in abnormal_types:
        nlv_per_anomaly_type[video_type] = int( math.ceil(len(os.listdir(videos_path+'/'+'1'+'/'+video_type)) * DoS/100.0) )
        nlv = nlv + nlv_per_anomaly_type[video_type]
    print("   Number of labeled videos: {}".format(nlv))

    # select video indices randomly from each type of videos based on the value of 'nlp_per_anomaly_type'
    labeled_video_samples = []
    unlabeled_video_samples = []
    labeled_samples_indices = []
    unlabeled_samples_indices = []
    actual_labels_of_labeled_samples_points = []
    actual_labels_of_unlabeled_samples_points = []

    dataset_1 = {}
    for i in range(totalInstances):
        dataset_1[dataset[i][0]] = dataset[i][1]
    abnormal_types.remove('Normal')
    for v_type in abnormal_types:
        videos_list = os.listdir(videos_path+'/'+'1'+'/'+ v_type)
        v_indices = random.sample( range(0,len(videos_list)), nlv_per_anomaly_type[v_type] )

        for v_index in v_indices:
            v_name = videos_list[v_index]
            for s_index in range(32):
                labeled_video_samples.append(dataset_1[v_type+"_"+v_name+"_"+str(s_index)])
                labeled_samples_indices.append(v_type+"_"+v_name+"_"+str(s_index))
                actual_labels_of_labeled_samples_points.append(labels[v_type+"_"+v_name+"_"+str(s_index)])
        for index in range(0,len(videos_list)):
            if index not in v_indices:
                v_name = videos_list[index]
                for s_index in range(32):
                    unlabeled_video_samples.append(dataset_1[v_type+"_"+v_name+"_"+str(s_index)])
                    unlabeled_samples_indices.append(v_type+"_"+v_name+"_"+str(s_index))
                    actual_labels_of_unlabeled_samples_points.append(labels[v_type+"_"+v_name+"_"+str(s_index)])
    PLP = len(labeled_video_samples)   # Number of positive labeled points
    videos_list = os.listdir(videos_path+'/'+'1'+'/'+ 'Normal')
    v_indices = random.sample( range(0,len(videos_list)), nlv_per_anomaly_type['Normal'] )
    for v_index in v_indices:
        v_name = videos_list[v_index]
        for s_index in range(32):
            labeled_video_samples.append(dataset_1['Normal'+"_"+v_name+"_"+str(s_index)])
            labeled_samples_indices.append('Normal'+"_"+v_name+"_"+str(s_index))
            actual_labels_of_labeled_samples_points.append(labels['Normal'+"_"+v_name+"_"+str(s_index)])
    for index in range(0,len(videos_list)):
        if index not in v_indices:
            v_name = videos_list[index]
            for s_index in range(32):
                unlabeled_video_samples.append(dataset_1['Normal'+"_"+v_name+"_"+str(s_index)])
                unlabeled_samples_indices.append('Normal'+"_"+v_name+"_"+str(s_index))
                actual_labels_of_unlabeled_samples_points.append(labels['Normal'+"_"+v_name+"_"+str(s_index)])
    
    NLP = len(labeled_video_samples) - PLP                        # Number of negative labeled points
    ULP = totalInstances - (PLP + NLP)                      # Number of unlabeled points
    return labeled_video_samples, unlabeled_video_samples, actual_labels_of_labeled_samples_points, actual_labels_of_unlabeled_samples_points, labeled_samples_indices, unlabeled_samples_indices, PLP, NLP, ULP

In [11]:
#=============================================================================================
"Function to find the frame level predicted labels for a given sgment level labels of a video"
#=============================================================================================
def find_frame_level_predicted_labels(segment_level_predictions,numberOfFrames):
    #print("SEGMENT LEVEL PREDICTIONS")
    #print(segment_level_predictions)
    NFrames=numberOfFrames
    ls = list(np.linspace(0,NFrames,33))
    ls = [int(round(x)) for x in ls]
    slp = segment_level_predictions
    flp = list(np.ones(NFrames)*-1)
    for i in range(32):
        if slp[i] >= 0.0:
            for j in range(ls[i],ls[i+1],1):
                flp[j] = 1.0
    #print(flp)
    return flp

In [12]:
#=============================================================================================================
"Function to find the frame level predicted labels for a given sgment level labels of all videos in test data"
#=============================================================================================================
def find_frame_level_predictions_for_testdata(predictions,segments_list_in_testdata, root_path_for_videos_directory, data_set):
  NVideos = int(len(predictions)/32)
  #print("   Number of segments for testing: {}".format(len(predictions)))
  #print("   Number of videos for tesing: {}".format(NVideos))
  FLresults = []
  NFrames = 0
  for i in range(NVideos):
    sname = segments_list_in_testdata[i*32] #Abnormal_Test001_video.txt_0
    if data_set == 'Ped2' or data_set == 'Ped1':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.avi')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')
    if data_set == 'MEM':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.mp4')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')
    if data_set == 'WS':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.mp4')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')

    v_path = root_path_for_videos_directory+v_type+'/'+v_name
    cap = cv2.VideoCapture(v_path)
    NFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    NFrames = int(NFrames)
    cap.release()
    if NFrames <= 1024:
      NFrames = NFrames + (1056 - NFrames)
    
    flresults = find_frame_level_predicted_labels(predictions[i*32:i*32+32],NFrames)
    FLresults = FLresults + flresults
  return FLresults

In [13]:
#=============================================================================================================
"Function to find the frame level true labels for a given testdata"
#=============================================================================================================
def find_frame_level_true_labels(temporal_annotations_file,segments_list_in_testdata,root_path_for_videos_directory,data_set):
  # get temporal information
  TA_file = temporal_annotations_file
  lines = open(TA_file,'r').readlines()
  TA_dict = {}
  for line in lines:
    w = line.strip().split(' ')
    TA_dict[w[0]] = w[1:]
  
  FLTrueLabels = []
  NVideos = int(len(segments_list_in_testdata)/32)

  NFrames = 0
  for i in range(NVideos):
    sname = segments_list_in_testdata[i*32] #Abnormal_Vandalism049_x264_C.txt_0
    if data_set == 'Ped1':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.avi')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')
    if data_set == 'Ped2':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.avi')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')
    if data_set == 'MEM':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.mp4')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')
    if data_set == 'WS':
      if sname.find('Normal') == -1:
        v_type = 'Abnormal'
        v_name = sname.replace('Abnormal_','')
      elif sname.find('Abnormal') == -1:
        v_type = 'Normal'
        v_name = sname.replace('Normal_','')
      else:
        print('SOMETHING WRONG WHILE GETING THE VIDEO FILE NAME FOR THE SEGMENT : {}'.format(sname))
        sys.exit()
      v_name = v_name.replace('.txt','.mp4')
      v_name = v_name.replace('_'+v_name.strip().split('_')[-1],'')
    v_path = root_path_for_videos_directory+v_type+'/'+v_name
    cap = cv2.VideoCapture(v_path)
    NFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    NFrames = int(NFrames)
    cap.release()
    if NFrames <= 1024:
      NFrames = NFrames + (1056 - NFrames)

    frame_level_true_labels_v = list(np.ones(NFrames) * -1)
    if int(TA_dict[v_type+'_'+v_name][0]) != -1 :
      l = len(TA_dict[v_type+'_'+v_name])
      index = 0
      while index < l:
        si = TA_dict[v_type+'_'+v_name][index]
        ei = TA_dict[v_type+'_'+v_name][index+1]
        for fn in range(int(si),min(int(ei),NFrames),1):
          try:
            frame_level_true_labels_v[fn] = 1
          except:
            print("WORNING: RUNTIME ERROR: IndexError")
            print("video name: "+v_name)
            print("current frame index to assign the label: {}".format(fn))
            print("Starting frame index: {}, Ending frame index: {} for current abnormal video portion".format(int(si),int(ei)))
            print("Actual number of frames in video folder: {}".format(NFrames))
        index = index+2
    FLTrueLabels = FLTrueLabels + frame_level_true_labels_v
  return FLTrueLabels

In [14]:
# This code is used to execute while finding the hyperparameters
def train_and_testing(videos_path, segment_level_labels_file_path, dataset, labels,root_path_for_videos_directory,data_set_name):
  parameters_list1 = [1.01] #[0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
  parameters_list2 = [1.01] #[0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
  parameters_list3 = [1.03] #[0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
  parameters_list4 = [0.9] #[0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
  lambda1 = 0.5
  lambda2a = 0.5
  lambda2b = 0.5
  lambda3 = 0.09 
  DoSs = [60.0] #[1.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 60.0, 70.0, 80.0]
  output = []
  for lambda1 in parameters_list1:
    for lambda2a in parameters_list2:
      for lambda2b in parameters_list3:
        for lambda3 in parameters_list4:
          for DoS in DoSs:
            iters = 20 #20 # No of times the model has to be run for each DoS value
            results = []
            results_f = []
            for it_num in range(iters):
              print("DoS: {}    Iter No: {} ".format(DoS,it_num))
              time_before = datetime.now()
              l = Labeled_and_Unlabeled_datapreparation(DoS, videos_path, segment_level_labels_file_path, dataset, labels)
              '''
              0 - labeled_video_samples
              1 - unlabeled_video_samples 
              2 - actual_labels_of_labeled_samples_points
              3 - actual_labels_of_unlabeled_samples_points
              4 - labeled_samples_indices
              5 - unlabeled_samples_indices
              6 - PLP
              7 - NLP
              8 - ULP
              '''
              print('   {}, {}, {}, {}, {}, {}, {}, {}, {}'.format(len(l[0]),len(l[1]),len(l[2]),len(l[3]),len(l[4]),len(l[5]),l[6],l[7],l[8]))
              #X_Train = np.float32(l[0])
              print("   Shape of labeled_video_samples: {}".format(np.shape(np.float32(l[0]))))
              #y_train = np.int8(l[2])
              print("   Shape of actual_labels_of_labeled_samples_points: {}".format(np.shape(np.int8(l[2]))))
              #X_Test = np.float32(l[1])
              print("   Shape of unlabeled_video_samples: {}".format(np.shape(np.float32(l[1]))))
              #y_test = np.int8(l[3])
              print("   Shape of actual_labels_of_unlabeled_samples_points: {}".format(np.shape(np.int8(l[3]))))
              print("   Number of positive labeld Instances: {}".format(l[6]))
              print("   Number of negative labeld Instances: {}".format(l[7]))
              print("   Number of unlabeld Instances: {}".format(l[8]))
              
              #"Construction of K-NN graph"
              print("   SIMILARITY WEIGHT MATRIX CONSTRUCTION...")
              dataset1 = l[0]+l[1]
              #print('length of l[0] : '.format(len(l[0])))
              #print('lenght of dataset is : '.format(len(dataset1)))
              neighbors = 6
              time_before = datetime.now() # starting time for graph construction
              nbrs_graph = NearestNeighbors(n_neighbors=neighbors, algorithm='ball_tree', metric = 'euclidean',n_jobs = 24).fit(np.array(dataset1))  #keyword argument 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']
              distances, indices = nbrs_graph.kneighbors(np.array(dataset1))
              neighbours_info = {}
              for i in range(len(dataset1)):
                nei = []
                for j in range(neighbors):
                  nei.append(indices[i][j])
                neighbours_info[i] = nei
              print("   SIMILARITY WEIGHT MATRIX CONSTRUCTION COMPLETED.")
              
              # sparce row matrix construction for weighted similarity matrix
              row = []
              col = []
              data = []
              for i in range(len(dataset)):
                for j in range(neighbors):
                  if i != indices[i][j]:
                    row.append(i)
                    col.append(indices[i][j])
                    data.append(distances[i][j])
              
              #finding connected components
              g = Graph(len(dataset))
              for i in range(len(dataset)):
                for j in range(neighbors):
                  g.addEdge(i, indices[i][j])
              
              cc = g.connectedComponents()
              print('   Number of Connected Components : {}.'.format(len(cc)))
              
              if len(cc) > 1 :
                for ci in range(len(cc)):
                  test_samples = [ dataset1[e] for e in cc[ci] if e< (l[6] + l[7]) ] # collect labeled instances
                  if len(test_samples) == 0:
                    continue
                  test_labeled_indices = [ e for e in cc[ci] if e< (l[6] + l[7]) ]
                  for cj in range(len(cc)):
                    if ci != cj:
                      X_temp = [dataset1[j]  for j in cc[cj]]
                      nbrs_graph = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric='euclidean', n_jobs = 24).fit(np.array(X_temp))  #keyword argument 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']
                      distance_s, indix_s = nbrs_graph.kneighbors(np.array(test_samples))
                      distance_s = np.squeeze(distance_s)
                      indix_s = np.squeeze(indix_s)
                      w = min(distance_s)
                      x_ind = list(distance_s).index(w)
                      source_v = test_labeled_indices[ list(distance_s).index(w)]
                      target_v = cc[cj][indix_s[x_ind]]
                      row.append(source_v)
                      col.append(target_v)
                      data.append(w)
                      g.addEdge(source_v, target_v)
                      neighbours_info[source_v].append(target_v)
                      #print('   Component {} and {} are connected with new edge <{}, {}>'.format(ci, cj, source_v, target_v))
                      
              cc = g.connectedComponents()
              print('   After adding edges number of Connected Components are : {}.'.format(len(cc)))
              
              # final csr_matrix creation after making single connected components
              total_data_points = l[6] + l[7] + l[8]
              W = csr_matrix((np.array(data), (np.array(row), np.array(col))), shape=(total_data_points, total_data_points))
              
              # final csr_graph creation
              csr_graph_of_W = csgraph_from_dense(W.toarray(), null_value=np.inf)
              print('   GRAPH ADJACENCY MATRIX CREATED')
              #print('CSR_Graph of W is : \n {}'.format(csr_graph_of_W))
              csr_GL = csgraph.laplacian(csr_graph_of_W, normed=True)
              print('   GRAPH LAPLACIAN MATRIX CREATED')
              #print('CSR_Graph Laplacian is :\n {}'.format(csr_GL))
              Iy = np.array(l[2] + l[3]).reshape(-1,32).tolist()
              L =int( (l[6] + l[7]) / 32)
              PL =int( l[6] /32)
              NL = int(l[7] / 32)
              n = l[6] + l[7] + l[8]
              N = int(n/32)
              
              #O = NCGssMILP(W.toarray(), neighbours_info, neighbors, Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N)
              O = NCGssMILP(W.toarray(),csr_GL, neighbours_info, Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N)
              predictions = O[l[6]+l[7]:]
              #print('Final output : {}'.format(predictions))
              for pv in range(len(predictions)):
                if predictions[pv] >= 0.0:
                  predictions[pv] = 1.0
                else:
                  predictions[pv] = -1.0
              # performance calculation in segment level
              # accuracy: (tp + tn) / (p + n)
              accuracy = accuracy_score(l[3], [round(e) for e in predictions])
              # precision tp / (tp + fp)
              precision = precision_score(l[3], [round(e) for e in predictions])
              # recall: tp / (tp + fn)
              recall = recall_score(l[3], [round(e) for e in predictions])
              # f1: 2 tp / (2 tp + fp + fn)
              f1 = f1_score(l[3], [round(e) for e in predictions])
              # ROC AUC
              auc = roc_auc_score(l[3], [round(e) for e in predictions])
              # confusion matrix
              matrix = confusion_matrix(l[3], [round(e) for e in predictions])
              #print(matrix)
              results.append([accuracy, precision, recall,f1,auc])
              print('Seg: acc - : {}, prec - : {}, rec - : {}, f1 - : {}, auc - : {}'.format(accuracy, precision, recall, f1, auc))
              
              # performance calculation in frame level
              #print("length of the pred and test samples {},{}".format(len(predictions),len(l[5])))
              flpredictions = find_frame_level_predictions_for_testdata(predictions,l[5],root_path_for_videos_directory,data_set_name)
              #print(flpredictions)
              fltrue_labels = find_frame_level_true_labels(frame_level_labels_file,l[5],root_path_for_videos_directory,data_set_name)
              #print(len(flpredictions),len(fltrue_labels))
              #print(flpredictions)
              # accuracy: (tp + tn) / (p + n)
              accuracy_f = accuracy_score(fltrue_labels, flpredictions)
              # precision tp / (tp + fp)
              precision_f = precision_score(fltrue_labels, flpredictions)
              # recall: tp / (tp + fn)
              recall_f = recall_score(fltrue_labels, flpredictions)
              # f1: 2 tp / (2 tp + fp + fn)
              f1_f = f1_score(fltrue_labels, flpredictions)
              # ROC AUC
              auc_f = roc_auc_score(fltrue_labels, flpredictions)
              # confusion matrix
              matrix_f = confusion_matrix(fltrue_labels, flpredictions)
              #print(matrix_f)
              results_f.append([accuracy_f, precision_f, recall_f, f1_f, auc_f])
              print('Frame: acc - : {}, prec - : {}, rec - : {}, f1 - : {}, auc - : {}'.format(accuracy_f, precision_f, recall_f, f1_f, auc_f))
              
            results = list(np.mean(np.array(results),axis=0))
            string1 = "   Classifier: GssMILP -  NN: {} lambda1: {} lambda2a: {} lambda2b: {} lambda3: {} DoS: {} RESULTS: ACC: {} PRE: {} RECALL: {} F1-SCORE: {} AUC: {}".format(neighbors, lambda1, lambda2a, lambda2b, lambda3, DoS, results[0], results[1], results[2], results[3], results[4])
            print(string1)
            
            results_f = list(np.mean(np.array(results_f),axis=0))
            string2 = "   Classifier: GssMILP -  NN: {} lambda1: {} lambda2a: {} lambda2b: {} lambda3: {} DoS: {} RESULTS_F: ACC: {} PRE: {} RECALL: {} F1-SCORE: {} AUC: {}".format(neighbors, lambda1, lambda2a, lambda2b, lambda3, DoS, results_f[0], results_f[1], results_f[2], results_f[3], results_f[4])
            print(string2)
            #output.append([string1,string2])       
  return output

In [ ]:
"""
def train_and_testing(videos_path, segment_level_labels_file_path, dataset, labels,root_path_for_videos_directory,data_set_name):
  lambda1 = 0.5
  lambda2a = 0.5
  lambda2b = 0.5
  lambda3 = 0.09 
  DoSs = [20.0]#[1.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]
  output = []
  for DoS in DoSs:
    iters = 1 #20 # No of times the model has to be run for each DoS value
    results = []
    results_f = []
    for it_num in range(iters):
      print("DoS: {}    Iter No: {} ".format(DoS,it_num))
      time_before = datetime.now()
      l = Labeled_and_Unlabeled_datapreparation(DoS, videos_path, segment_level_labels_file_path, dataset, labels)
      '''
      0 - labeled_video_samples
      1 - unlabeled_video_samples 
      2 - actual_labels_of_labeled_samples_points
      3 - actual_labels_of_unlabeled_samples_points
      4 - labeled_samples_indices
      5 - unlabeled_samples_indices
      6 - PLP
      7 - NLP
      8 - ULP
      '''
      print('   {}, {}, {}, {}, {}, {}, {}, {}, {}'.format(len(l[0]),len(l[1]),len(l[2]),len(l[3]),len(l[4]),len(l[5]),l[6],l[7],l[8]))
      #X_Train = np.float32(l[0])
      print("   Shape of labeled_video_samples: {}".format(np.shape(np.float32(l[0]))))
      #y_train = np.int8(l[2])
      print("   Shape of actual_labels_of_labeled_samples_points: {}".format(np.shape(np.int8(l[2]))))
      #X_Test = np.float32(l[1])
      print("   Shape of unlabeled_video_samples: {}".format(np.shape(np.float32(l[1]))))
      #y_test = np.int8(l[3])
      print("   Shape of actual_labels_of_unlabeled_samples_points: {}".format(np.shape(np.int8(l[3]))))
      print("   Number of positive labeld Instances: {}".format(l[6]))
      print("   Number of negative labeld Instances: {}".format(l[7]))
      print("   Number of unlabeld Instances: {}".format(l[8]))

      #"Construction of K-NN graph"
      print("   SIMILARITY WEIGHT MATRIX CONSTRUCTION...")
      dataset1 = l[0]+l[1]
      #print('length of l[0] : '.format(len(l[0])))
      #print('lenght of dataset is : '.format(len(dataset1)))
      neighbors = 6
      time_before = datetime.now() # starting time for graph construction
      nbrs_graph = NearestNeighbors(n_neighbors=neighbors, algorithm='ball_tree', metric = 'euclidean',n_jobs = 24).fit(np.array(dataset1))  #keyword argument 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']
      distances, indices = nbrs_graph.kneighbors(np.array(dataset1))
      #print("Time taken for K-NN graph construction: {}".format(datetime.now() - time_before))
      #print(distances)
      #print(type(indices))
      #print(indices)
      # indices dict
      neighbours_info = {}
      for i in range(len(dataset1)):
        nei = []
        for j in range(neighbors):
          nei.append(indices[i][j])
        neighbours_info[i] = nei
      print("   SIMILARITY WEIGHT MATRIX CONSTRUCTION COMPLETED.")

      # sparce row matrix construction for weighted similarity matrix
      row = []
      col = []
      data = []
      for i in range(len(dataset)):
        for j in range(neighbors):
          if i != indices[i][j]:
            row.append(i)
            col.append(indices[i][j])
            data.append(distances[i][j])
      

      #finding connected components
      g = Graph(len(dataset))
      for i in range(len(dataset)):
        for j in range(neighbors):
          g.addEdge(i, indices[i][j])

      cc = g.connectedComponents() 
      print('   Number of Connected Components : {}.'.format(len(cc)))
      #print("Following are connected components")
      #for c in range(len(cc)):
      #  print(cc[c])
      if len(cc) > 1 :
        for ci in range(len(cc)):
          test_samples = [ dataset1[e] for e in cc[ci] if e< (l[6] + l[7]) ] # collect labeled instances
          if len(test_samples) == 0:
            #print('*** The connected component {} does not has labeled samples **'.format(ci))
            #print('Nodes in this component is :\n {} '.format(cc[ci]))
            continue
          else:
            #print('   *** The connected component {} has labeled samples **'.format(ci))
            pass
          test_labeled_indices = [ e for e in cc[ci] if e< (l[6] + l[7]) ]
          for cj in range(len(cc)):
            if ci != cj:
              X_temp = [dataset1[j]  for j in cc[cj]]
              nbrs_graph = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric='euclidean', n_jobs = 24).fit(np.array(X_temp))  #keyword argument 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']
              distance_s, indix_s = nbrs_graph.kneighbors(np.array(test_samples))
              distance_s = np.squeeze(distance_s)
              indix_s = np.squeeze(indix_s)
              w = min(distance_s)
              x_ind = list(distance_s).index(w)
              source_v = test_labeled_indices[ list(distance_s).index(w)]
              target_v = cc[cj][indix_s[x_ind]]
              row.append(source_v)
              col.append(target_v)
              data.append(w)
              g.addEdge(source_v, target_v)
              neighbours_info[source_v].append(target_v)
              #print('   Component {} and {} are connected with new edge <{}, {}>'.format(ci, cj, source_v, target_v))

      cc = g.connectedComponents() 
      #print("Following are connected components")
      #print(cc)
      print('   After adding edges number of Connected Components are : {}.'.format(len(cc)))

      # final csr_matrix creation after making single connected components
      total_data_points = l[6] + l[7] + l[8]
      W = csr_matrix((np.array(data), (np.array(row), np.array(col))), shape=(total_data_points, total_data_points))
      #print(W.toarray())
      #print('The size of W is {}'.format(W.shape))

      # final csr_graph creation
      csr_graph_of_W = csgraph_from_dense(W.toarray(), null_value=np.inf)
      print('   GRAPH ADJACENCY MATRIX CREATED')
      #print('CSR_Graph of W is : \n {}'.format(csr_graph_of_W))
      csr_GL = csgraph.laplacian(csr_graph_of_W, normed=True)
      print('   GRAPH LAPLACIAN MATRIX CREATED')
      #print('CSR_Graph Laplacian is :\n {}'.format(csr_GL))
      Iy = np.array(l[2] + l[3]).reshape(-1,32).tolist()
      L =int( (l[6] + l[7]) / 32)
      PL =int( l[6] /32)
      NL = int(l[7] / 32)
      n = l[6] + l[7] + l[8]
      N = int(n/32)

      #O = NCGssMILP(csr_GL.toarray(), Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N)
      #O = NCGssMILP(W.toarray(), indices, neighbors, Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N)
      #O = NCGssMILP(W.toarray(), neighbours_info, neighbors, Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N)
      O = NCGssMILP(W.toarray(),csr_GL, neighbours_info, Iy, lambda1, lambda2a, lambda2b, lambda3, L,PL, NL, n, N)
      predictions = O[l[6]+l[7]:]
      #print('Final output : {}'.format(predictions))
      for pv in range(len(predictions)):
        if predictions[pv] >= 0.0:
          predictions[pv] = 1.0
        else:
          predictions[pv] = -1.0
      # performance calculation in segment level
      # accuracy: (tp + tn) / (p + n)
      accuracy = accuracy_score(l[3], [round(e) for e in predictions])
      # precision tp / (tp + fp)
      precision = precision_score(l[3], [round(e) for e in predictions])
      # recall: tp / (tp + fn)
      recall = recall_score(l[3], [round(e) for e in predictions])
      # f1: 2 tp / (2 tp + fp + fn)
      f1 = f1_score(l[3], [round(e) for e in predictions])
      # ROC AUC
      auc = roc_auc_score(l[3], [round(e) for e in predictions])
      # confusion matrix
      matrix = confusion_matrix(l[3], [round(e) for e in predictions])
      #print(matrix)
      results.append([accuracy, precision, recall,f1,auc])

      # performance calculation in frame level
      #print("length of the pred and test samples {},{}".format(len(predictions),len(l[5])))
      flpredictions = find_frame_level_predictions_for_testdata(predictions,l[5],root_path_for_videos_directory,data_set_name)
      #print(flpredictions)
      fltrue_labels = find_frame_level_true_labels(frame_level_labels_file,l[5],root_path_for_video_frames_directory,data_set_name)
      #print(len(flpredictions),len(fltrue_labels))
      #print(flpredictions)
      # accuracy: (tp + tn) / (p + n)
      accuracy_f = accuracy_score(fltrue_labels, flpredictions)
      # precision tp / (tp + fp)
      precision_f = precision_score(fltrue_labels, flpredictions)
      # recall: tp / (tp + fn)
      recall_f = recall_score(fltrue_labels, flpredictions)
      # f1: 2 tp / (2 tp + fp + fn)
      f1_f = f1_score(fltrue_labels, flpredictions)
      # ROC AUC
      auc_f = roc_auc_score(fltrue_labels, flpredictions)
      # confusion matrix
      matrix_f = confusion_matrix(fltrue_labels, flpredictions)
      #print(matrix_f)
      results_f.append([accuracy_f, precision_f, recall_f,f1_f,auc_f])

    results = list(np.mean(np.array(results),axis=0))
    string1 = "   Classifier: GssMILP NN: {} lambda1: {} lambda2: {} lambda3: {} DoS: {} RESULTS: ACC: {} PRE: {} RECALL: {} F1-SCORE: {} AUC: {}".format(neighbors, lambda_1, lambda_2,lambda_4, DoS,results[0],results[1],results[2],results[3],results[4])
    print(string1)

    results_f = list(np.mean(np.array(results_f),axis=0))
    string2 = "   Classifier: Deep_GMIL NN: {} lambda1: {} lambda2: {} lambda3: {} DoS: {} RESULTS_F: ACC: {} PRE: {} RECALL: {} F1-SCORE: {} AUC: {}".format(neighbors, lambda_1, lambda_2,lambda_4, DoS,results_f[0],results_f[1],results_f[2],results_f[3],results_f[4])
    print(string2)
    output.append([string1,string2])       
  return output """

In [ ]:
# creating segment level labels file. 

frame_level_labels_file = "./dataset/Temporal_Anomaly_Annotation_for_full_data_of_MEM.txt" 
root_path_for_videos_directory = './MEM_videos/'
output_file = "./dataset/segment_level_labels_MEM.txt"
data_set = 'MEM'
find_segment_level_labels(frame_level_labels_file, root_path_for_videos_directory, output_file,data_set)
print("SEGMENT LEVEL LABELS FILE IS CREATED")

# Creating Dictionary of features (of size N*4096) and Dictionary of labels (of size N*1)
videos_path="./dataset/MEM_dataset_1"#MEM_dataset_1/   Ped1_dataset_1/   WS_dataset_1/
segment_level_labels_file_path = "./dataset/segment_level_labels_MEM.txt"
features_option = 3 # either 1 or 2 or 3. 1: only C3D features considered. 2: only C3D_BICLSTM features considered. 3: Both features are considered.
dataset, labels = combining_video_features(videos_path, segment_level_labels_file_path,features_option)
print("Dataset Inf: Number of instances - {}, Number of labes {}".format(len(dataset),len(labels)))
print('Lenth of feature vector: {}'.format(len(dataset[0][1])))
print("COMBINING VIDEO FEATURES AND PREPARATION OF DATASET IS COMPLETED.")
train_and_testing(videos_path, segment_level_labels_file_path, dataset, labels, root_path_for_videos_directory,data_set)

# RESULTS

In [17]:
cd /content/drive/My Drive/NSK/GssMILP_Results

/content/drive/My Drive/NSK/GssMILP_Results


In [18]:
!cat MEM_results_on_C3D_features.txt | grep -E 'Classifier'

   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 1.0 RESULTS: ACC: 0.9280172413793103 PRE: 0.780752375115761 RECALL: 0.9497436246440456 F1-SCORE: 0.8569802494249539 AUC: 0.9356933187937928
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 1.0 RESULTS_F: ACC: 0.902384149456207 PRE: 0.7080606862927661 RECALL: 0.9227835637947784 F1-SCORE: 0.8012105914038268 AUC: 0.9098476450835682
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 5.0 RESULTS: ACC: 0.926293103448276 PRE: 0.7741214616150156 RECALL: 0.9460045934399618 F1-SCORE: 0.851454012887444 AUC: 0.9333145750873975
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 5.0 RESULTS_F: ACC: 0.9002806378118515 PRE: 0.7033320183437549 RECALL: 0.9158421905027143 F1-SCORE: 0.7955941170728674 AUC: 0.9059832258927379
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01

In [19]:
!cat MEM_results_on_C3D_BICLSTM_features.txt | grep -E 'Classifier'

   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 1.0 RESULTS: ACC: 0.18405172413793106 PRE: 0.09728916955220442 RECALL: 0.33112699153818287 F1-SCORE: 0.15033988898999953 AUC: 0.23712699693912082
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 1.0 RESULTS_F: ACC: 0.22002010516782672 PRE: 0.0998367397935298 RECALL: 0.3464399138521538 F1-SCORE: 0.15496993903767173 AUC: 0.2668118217686142
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 5.0 RESULTS: ACC: 0.18469827586206894 PRE: 0.09979076000514563 RECALL: 0.3223760463440323 F1-SCORE: 0.15235897838756146 AUC: 0.2333087225541745
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 5.0 RESULTS_F: ACC: 0.2193822313437152 PRE: 0.10062253585730671 RECALL: 0.3360188852045935 F1-SCORE: 0.15483245038166188 AUC: 0.2619274067101607
   Classifier: GssMILP -  NN: 6 lambda1: 1.0

In [20]:
!cat MEM_results_on_both_features.txt | grep -E 'Classifier'

   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 1.0 RESULTS: ACC: 0.18189655172413793 PRE: 0.09710809097294906 RECALL: 0.3197404863951749 F1-SCORE: 0.14893104242484084 AUC: 0.23093896468877628
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 1.0 RESULTS_F: ACC: 0.19781711477488 PRE: 0.09805065764811008 RECALL: 0.3399771899042021 F1-SCORE: 0.15217647815247035 AUC: 0.24978100440016124
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 5.0 RESULTS: ACC: 0.1842133620689655 PRE: 0.09873133771372002 RECALL: 0.32250102134173947 F1-SCORE: 0.1511281628437406 AUC: 0.23328043793953834
   Classifier: GssMILP -  NN: 6 lambda1: 1.01 lambda2a: 1.01 lambda2b: 1.03 lambda3: 0.9 DoS: 5.0 RESULTS_F: ACC: 0.19967913993211556 PRE: 0.09936222999500784 RECALL: 0.3427555336896061 F1-SCORE: 0.15402689092750016 AUC: 0.25191096969691956
   Classifier: GssMILP -  NN: 6 lambda1: 1.0

In [ ]:
cd /content/drive/My Drive/AnomalyDetection-v-a

/content/drive/My Drive/AnomalyDetection-v-a


In [ ]:
"""
videos_path = './datasets/WS_dataset/videos'
total_frames = 0
folder_names = os.listdir(videos_path)
video_names = []
for f in folder_names:
  video_names = video_names + [f+'/'+v for v in os.listdir(videos_path+'/'+f)]
#video_names = os.listdir(videos_path)
for v_name in video_names:
  v_path = videos_path+'/'+v_name
  #print(v_name)
  cap = cv2.VideoCapture(v_path)
  no_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  no_of_frames = int(no_of_frames)
  string = '{} {}'.format(v_name,no_of_frames)
  print(string)
  total_frames = total_frames + no_of_frames
  cap.release()

TA_file_path = './datasets/Temporal_Annotations_for_full_data_of_WS.txt'
lines = open(TA_file_path,'r').readlines()
total_positive_labeled_frames = 0
for line in lines:
  #print(line)
  w = line.strip().split(' ')
  #print(w)
  labeled_frames = 0
  length_of_the_line = len(w)
  #print(length_of_the_line)
  i = 1
  if int(w[1]) == -1:
    #normal video
    pass
  else:
    while i < length_of_the_line-1:
      labeled_frames = labeled_frames + (int(w[i+1]) - int(w[i]) + 1)
      i = i+1
  print('{} {}'.format(w[0],labeled_frames))
  total_positive_labeled_frames = total_positive_labeled_frames + labeled_frames

print('total frames: {}, total labeled frames: {}, CI: {}'.format(total_frames,total_positive_labeled_frames,(total_frames - total_positive_labeled_frames)*1.0/total_frames *100.0))
"""
